In [2]:
import pandas as pd
import numpy as np
import math
from functools import reduce
import difflib
import copy
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
import matplotlib.pyplot as plt

In [4]:
# Load the datasets
df_successful = [pd.read_excel (r'Data/successful.xlsx', sheet_name= sheetname) for sheetname in ['List', 'Academic', 'Work']]
df_brand = [pd.read_excel (r'Data/brand.xlsx', sheet_name= sheetname) for sheetname in ['List', 'Academic', 'Work']]
df_failed = [pd.read_excel (r'Data/failed.xlsx', sheet_name= sheetname) for sheetname in ['List', 'Academic', 'Work']]

In [6]:
# Change column names
df_successful[0]=df_successful[0].rename(columns = {'name':'org_name'})
df_successful[2]=df_successful[2].rename(columns = {'organization_name':'org_name'})
df_brand[0]=df_brand[0].rename(columns = {'name':'org_name'})
df_brand[2]=df_brand[2].rename(columns = {'organization_name':'org_name'})
df_failed[0]=df_failed[0].rename(columns = {'name':'org_name'})
df_failed[2]=df_failed[2].rename(columns = {'organization_name':'org_name'})

In [7]:
# For rach dataset, merge the sheets into a single dataframe
df_successful_merged = reduce(lambda left,right: pd.merge(left,right,on=['org_name'],how='inner'),
                                       df_successful)
# add success flag variable. 0,1,2 for failed, succeessful and brand investors respectively
df_successful_merged['success_flag'] = 1

df_brand_merged = reduce(lambda left,right: pd.merge(left,right,on=['org_name'],how='inner'),
                                       df_brand)
df_brand_merged['success_flag'] = 2

df_failed_merged = reduce(lambda  left,right: pd.merge(left,right,on=['org_name'],how='inner'),
                                       df_failed)
df_failed_merged['success_flag'] = 0

In [8]:
# merge all dataframes, only including organizations with data in all sheets
df_all_merged = pd.concat([df_successful_merged, df_brand_merged, df_failed_merged], ignore_index=True, sort=False)


df_all_merged

,org_name,domain,founded_on,city,category_list,category_groups_list,short_description,universities_of_founders,degrees_of_founders,subject_degrees_of_founders,gender_of_founders,city_of_founders,prev_companies_of_founders,prev_title_of_founders,success_flag
0,Piictu,piictu.com,2011-02-01,New York,"Crowdsourcing,Finance,Mobile,Social Media","Financial Services,Internet Services,Media and...",Piictu is an influence-based photo gaming tool...,"Bentley University,Central University of Venez...","BA,BS,BS,BS,Reaserch Program","Architecture,Computer Science,Design,Economics...","male,male,male,male,male","Miami,Miami,New York,New York,New York","Alter,Blue Apron,Boardriders,Central Universit...","Advisor,COO,Chief Product Officer / Co-founder...",1
1,Tag (Acquired),gettagapp.com,2014-07-01,New York,"Apps,Mobile,Private Social Networking","Apps,Community and Lifestyle,Mobile,Software",Tag is your digital passport. (Acquired 2015),"Columbia University,Rutgers University,Rutgers...","B.S.,BS","Computer Science,Economics,Mathematics","male,male,male","Los Angeles,Philadelphia,San Francisco","Community,Community,ODIN,Renew Health,Soma Cap...","Co-Founder,Co-Founder,Co-Founder & CTO,Founder...",1
2,Skylight,skylightapp.com,2014-01-01,New York,"Apps,Real Estate,Smart Home","Apps,Consumer Electronics,Real Estate,Software",HomeSwipe is an iOS and Android app that enabl...,"Harvard Business School,INSEAD,NYU Stern Schoo...","BS,BscE,MBA,unknown,unknown","Business,Civil & Environmental Engineering,Fin...","female,female,male,male,male","New York,Toronto,Toronto","Appointment Status,BloomCredit,Bloomspot,Build...","Advisor,CEO,CTO / Co-Founder,Co-Founder,Co-Fou...",1
3,Skylight,skylight.com,2016-01-01,San Francisco,"Construction,Home Renovation,Internet,Project ...","Administrative Services,Internet Services,Othe...",Skylight is a Fully integrated and technology-...,"Harvard Business School,INSEAD,NYU Stern Schoo...","BS,BscE,MBA,unknown,unknown","Business,Civil & Environmental Engineering,Fin...","female,female,male,male,male","New York,Toronto,Toronto","Appointment Status,BloomCredit,Bloomspot,Build...","Advisor,CEO,CTO / Co-Founder,Co-Founder,Co-Fou...",1
4,OrderAhead,orderaheadapp.com,2011-01-01,San Francisco,"E-Commerce,Local,Mobile,Mobile Payments,Paymen...","Commerce and Shopping,Financial Services,Food ...",OrderAhead powers mobile ordering for pickup a...,"Massachusetts Institute of Technology,Stanford...","BS,MBA","Business Development,Computer Science","male,male","Menlo Park,San Francisco","BLADE,OrderAhead,OrderAhead","CEO and Co-Founder,Co-Founder, CEO,Co-Founder,...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8444,Leanplum,leanplum.com,2012-01-01,San Francisco,"A/B Testing,CRM,E-Commerce Platforms,Email Mar...","Artificial Intelligence,Commerce and Shopping,...",Leanplum is a mobile marketing platform that d...,"Carnegie Mellon University,Carnegie Mellon Uni...","B.S,B.S,B.S,B.S,M.S,M.S","Computer Science, Mathematics, Computational S...","male,male,male,male,male,male","San Francisco,San Francisco,San Francisco,San ...","Bluetunes.net,Bluetunes.net,Google,Google,Goog...","Co-Founder,Co-Founder,Co-Founder & CTO,Co-Foun...",0
8445,Playvox,playvox.com,2012-01-01,Sunnyvale,"Application Performance Management,Customer Se...","Data and Analytics,Other,Professional Services...","Playvox, the leading provider of workforce eng...","Universidad Autónoma de Manizales,Universidad ...","Degree,Degree","Systems Engineer,Systems Engineer","male,male","Manizales,Manizales","Arcaris,Arcaris,Playvox,Playvox,Playvox,Playvox","Founder & CEO,Founder & CEO,Founder & CEO,Foun...",0
8446,Dollar Shave Club,dollarshaveclub.com,2012-01-01,Santa Monica,"E-Commerce,Internet,Retail","Commerce and Shopping,Internet Services",DSC is a lifestyle brand and e-commerce compan...,Emory University,Bachelor's degree in arts,History,male,Santa Monica,"Dollar Shave Club,Dollar Shave Club,Dollar Sha...","Advisor,Associate Producer,Board Of Directo

In [9]:
# Check flag count and headings 
df_all_merged.info()
df_all_merged['success_flag'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8449 entries, 0 to 8448
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   org_name                     8449 non-null   object        
 1   domain                       8449 non-null   object        
 2   founded_on                   8449 non-null   datetime64[ns]
 3   city                         8449 non-null   object        
 4   category_list                8383 non-null   object        
 5   category_groups_list         8383 non-null   object        
 6   short_description            8449 non-null   object        
 7   universities_of_founders     8449 non-null   object        
 8   degrees_of_founders          8256 non-null   object        
 9   subject_degrees_of_founders  8034 non-null   object        
 10  gender_of_founders           8443 non-null   object        
 11  city_of_founders             7808 non-null 

0    4989
1    2030
2    1430
Name: success_flag, dtype: int64

In [17]:
# Import university rankings
df_University_Rankings = pd.read_excel(r'Data/2022_QS_World_University_Rankings_Results_public_version_modified.xlsx')
df_University_Rankings

,RANK_2022,Institution_Name,CODE,COUNTRY / TERRITORY,Academic Reputation,Employer Reputation,Faculty Student,Citations per Faculty,International Faculty,International Students,Overall
0,1,Massachusetts Institute of Technology (MIT),US,United States,100.0,100.0,100.0,100.0,100.0,91.4,100.0
1,2,University of Oxford,UK,United Kingdom,100.0,100.0,100.0,96.0,99.5,98.5,99.5
2,3,Stanford University,US,United States,100.0,100.0,100.0,99.9,99.8,67.0,98.7
3,4,University of Cambridge,UK,United Kingdom,100.0,100.0,100.0,92.1,100.0,97.7,98.7
4,5,Harvard University,US,United States,100.0,100.0,99.1,100.0,84.2,70.1,98.0
...,...,...,...,...,...,...,...,...,...,...,...
1295,1296,University POLITEHNICA of Bucharest,RO,Romania,6.4,5.6,4.8,7.0,1.1,3.2,20.0
1296,1297,"University Politehnica of Timisoara, UPT",RO,Romania,3.3,2.9,10.6,3.9,1.0,2.1,20.0
1297,1298,Yarmouk University,JO,Jordan,9.3,10.2,1.9,2.2,1.2,11.0,20.0
1298,1299,Yildiz Technical University,TR,Turkey,5.6,10.5,6.1,5.4,1.1,8.9,20.0


In [18]:
# List of all universities attended by founders
temp = df_all_merged['universities_of_founders'].values[:].tolist()
list_all_universities_of_founders = [x.split(',') for x in temp if isinstance(x, str)]
list_all_universities_of_founders = reduce(lambda x,y :x+y ,list_all_universities_of_founders)
list_all_universities_of_founders = list(set(list_all_universities_of_founders))

In [ ]:
default_value = -1
default_country = 'NA'
minumum_overall_score = 20 
minimum_employer_score = 1
list_all_universities_of_founders_scores = [[x, default_value, default_value, default_country] for x in list_all_universities_of_founders]
 
# match universities attended by founders to universities in ranking with: overall score/ employer reputation score/ country code
for idx1 in range(len(list_all_universities_of_founders)):
    for idx2 in range(len(df_University_Rankings['Institution_Name'].values[:])):
        if list_all_universities_of_founders[idx1] in df_University_Rankings['Institution_Name'].values[idx2] and list_all_universities_of_founders_scores[idx1][1] == default_value:
            list_all_universities_of_founders_scores[idx1][1] = df_University_Rankings.at[idx2, 'Overall']
            list_all_universities_of_founders_scores[idx1][2] = df_University_Rankings.at[idx2, 'Employer Reputation']
            list_all_universities_of_founders_scores[idx1][3] = df_University_Rankings.at[idx2, 'CODE']
         
removal_strings = ['','university', 'University', 'university of', 'University of', 'University Of']

for name in removal_strings:
    for idx1 in range(len(df_University_Rankings['Institution_Name'].values[:])):
        for idx2 in range(len(list_all_universities_of_founders)):        
            if df_University_Rankings['Institution_Name'].values[idx1].replace(name,'').strip() in list_all_universities_of_founders[idx2] and list_all_universities_of_founders_scores[idx2][1] == default_value:
                list_all_universities_of_founders_scores[idx2][1] = df_University_Rankings.at[idx1, 'Overall']
                list_all_universities_of_founders_scores[idx1][2] = df_University_Rankings.at[idx1, 'Employer Reputation']
                list_all_universities_of_founders_scores[idx1][3] = df_University_Rankings.at[idx1, 'CODE']

for idx1 in range(len(list_all_universities_of_founders)):
    temp = difflib.get_close_matches(list_all_universities_of_founders[idx1],df_University_Rankings['Institution_Name'].values[:])
    if temp and list_all_universities_of_founders_scores[idx2][1] == default_value:
        list_all_universities_of_founders_scores[idx1][1] = df_University_Rankings.loc[df_University_Rankings['Institution_Name'] == temp[0], 'Overall'].values[0]
        list_all_universities_of_founders_scores[idx1][2] = df_University_Rankings.loc[df_University_Rankings['Institution_Name'] == temp[0], 'Employer Reputation'].values[0]
        list_all_universities_of_founders_scores[idx1][3] = df_University_Rankings.loc[df_University_Rankings['Institution_Name'] == temp[0], 'CODE'].values[0]
        
for name in removal_strings:
    for idx1 in range(len(df_University_Rankings['Institution_Name'].values[:])):
        temp = difflib.get_close_matches(df_University_Rankings['Institution_Name'].values[idx1].replace(name,'').strip(),list_all_universities_of_founders)
        for elem in temp: 
            idx2 = list_all_universities_of_founders.index(elem)
            if list_all_universities_of_founders_scores[idx2][1] == default_value:
                list_all_universities_of_founders_scores[idx2][1] = df_University_Rankings.at[idx1, 'Overall']
                list_all_universities_of_founders_scores[idx2][2] = df_University_Rankings.at[idx1, 'Employer Reputation']
                list_all_universities_of_founders_scores[idx2][3] = df_University_Rankings.at[idx1, 'CODE']

for x in list_all_universities_of_founders_scores:
    if x[1] == -1:
        x[1] = minumum_overall_score
    if x[2] == -1:
        x[2]= minimum_employer_score

In [14]:
# Count the following features: 
# university score, employer score, geographical features, different universities (Stanford, MIT, IVY)

temp = df_all_merged['universities_of_founders'].values[:]

df_all_merged['maximum_founders_university_score'] = float('nan')
df_all_merged['minimum_founders_university_score'] = float('nan')
df_all_merged['average_founders_university_score'] = float('nan')
df_all_merged['average_founders_university_employer_score'] = float('nan')
df_all_merged['maximum_founders_university_employer_score'] = float('nan')
df_all_merged['uni_country'] = float('nan')
df_all_merged['uni_country'] = df_all_merged['uni_country'] .astype(object)
df_all_merged['US_uni_flag'] = float('nan')
df_all_merged['US_uni_percentage'] = float('nan')
df_all_merged['stanford_flag'] = float('nan')
df_all_merged['stanford_percentage'] = float('nan')
df_all_merged['MIT_flag'] = float('nan')
df_all_merged['MIT_percentage'] = float('nan')
df_all_merged['IVY_flag'] = float('nan')
df_all_merged['IVY_percentage'] = float('nan')
df_all_merged['top_int_flag'] = float('nan')
df_all_merged['top_int_percentage'] = float('nan')

# top 3 uni from each continent according to QS 2023 rankings
top_int_list = ['University of Oxford', 'University of Cambridge', 'Imperial College London',
                'National University of Singapore (NUS)', 'Peking University', 
                'Nanyang Technological University, Singapore (NTU)', 'King Abdulaziz University (KAU)', 
                'Qatar University', 'King Fahd University of Petroleum & Minerals',
               'Pontificia Universidad Católica de Chile (UC)', 'Universidade de São Paulo', 'Universidad de Chile',
               'Lomonosov Moscow State University', 'Charles University', 'Saint Petersburg State University',
               'Tsinghua University', 'Fudan University', 'Australian National University', 'The University of Melbourne', 
                'The University of Sydney', 'Universidad de Buenos Aires (UBA)', 'Universidad Nacional Autónoma de México (UNAM)',
               'Universidade de São Paulo']

IVY_list = ['Harvard University', 'University of Pennsylvania', 'Princeton University', 'Yale University', 'Cornell University',
           'Columbia University', 'Brown University', 'Dartmouth College']

for idx in range(len(temp)):
    if not isinstance(temp[idx], str):
        if math.isnan(temp[idx]):
            continue
    temp_university_names = temp[idx].split(',')
    temp_university_rankings = []
    temp_university_scores = []
    temp_university_employer_scores = []
    temp_uni_country = []
    temp_us_count = 0
    temp_stanford_count = 0
    temp_MIT_count = 0
    temp_IVY_count = 0
    temp_top_int_count = 0
    
    for university_name in temp_university_names:
        idx2 = list_all_universities_of_founders.index(university_name)
        temp_university_scores.append(list_all_universities_of_founders_scores[idx2][1])
        temp_university_employer_scores.append(list_all_universities_of_founders_scores[idx2][2])
        temp_uni_country.append(list_all_universities_of_founders_scores[idx2][3])
        if list_all_universities_of_founders_scores[idx2][3] == 'US':
            temp_us_count += 1
        if list_all_universities_of_founders_scores[idx2][0] == 'Stanford University':
            temp_stanford_count += 1
        if 'MIT' in list_all_universities_of_founders_scores[idx2][0]:
            if list_all_universities_of_founders_scores[idx2][0] == 'RMIT University':
                continue 
            else:
                temp_MIT_count += 1
        if list_all_universities_of_founders_scores[idx2][0] in IVY_list:
            temp_IVY_count += 1
        if list_all_universities_of_founders_scores[idx2][0] in top_int_list:
            temp_top_int_count += 1
            
    
    df_all_merged.at[idx, 'num_universities'] = len(temp_university_names)
    df_all_merged.at[idx, 'maximum_founders_university_score'] = np.max(np.array(temp_university_scores))
    df_all_merged.at[idx, 'minimum_founders_university_score'] = np.min(np.array(temp_university_scores))
    df_all_merged.at[idx, 'average_founders_university_score'] = np.average(np.array(temp_university_scores))
    df_all_merged.at[idx, 'average_founders_university_employer_score'] = np.average(np.array(temp_university_employer_scores))
    df_all_merged.at[idx, 'maximum_founders_university_employer_score'] = np.max(np.array(temp_university_employer_scores))
    
    df_all_merged.at[idx, 'uni_country'] = temp_uni_country
    
    if temp_us_count > 0:
        df_all_merged.at[idx, 'US_uni_flag'] = 1
    else:
        df_all_merged.at[idx, 'US_uni_flag'] = 0
    df_all_merged.at[idx, 'US_uni_percentage'] = temp_us_count / len(temp[idx])
    
    if temp_stanford_count > 0:
        df_all_merged.at[idx, 'stanford_flag'] = 1
    else:
        df_all_merged.at[idx, 'stanford_flag'] = 0
    df_all_merged.at[idx, 'stanford_percentage'] = temp_stanford_count / len(temp[idx])
    
    if temp_MIT_count > 0:
        df_all_merged.at[idx, 'MIT_flag'] = 1
    else:
        df_all_merged.at[idx, 'MIT_flag'] = 0
    df_all_merged.at[idx, 'MIT_percentage'] = temp_MIT_count / len(temp[idx])
    
    if temp_IVY_count > 0:
        df_all_merged.at[idx, 'IVY_flag'] = 1
    else:
        df_all_merged.at[idx, 'IVY_flag'] = 0
    df_all_merged.at[idx, 'IVY_percentage'] = temp_IVY_count / len(temp[idx])
   
    if temp_top_int_count > 0:
        df_all_merged.at[idx, 'top_int_flag'] = 1
    else:
        df_all_merged.at[idx, 'top_int_flag'] = 0
    df_all_merged.at[idx, 'top_int_percentage'] = temp_top_int_count / len(temp[idx])
    

In [23]:
# Check some features
df_all_merged[['uni_country','US_uni_flag','stanford_percentage', 'MIT_flag', 'IVY_flag', 'num_universities', 'maximum_founders_university_score',
               'minimum_founders_university_score', 'average_founders_university_score',  'average_founders_university_employer_score']]

,uni_country,US_uni_flag,stanford_percentage,MIT_flag,IVY_flag,num_universities,maximum_founders_university_score,minimum_founders_university_score,average_founders_university_score,average_founders_university_employer_score
0,"[US, DK, NA, US, VE]",1.0,0.000000,0.0,0.0,5.0,59.9,20.0,29.34,12.780000
1,"[US, US, US]",1.0,0.000000,0.0,1.0,3.0,88.7,20.0,42.90,38.000000
2,"[NA, NA, US, US, UK]",1.0,0.000000,0.0,1.0,5.0,98.0,20.0,54.76,28.680000
3,"[NA, NA, US, US, UK]",1.0,0.000000,0.0,1.0,5.0,98.0,20.0,54.76,28.680000
4,"[US, US]",1.0,0.017544,0.0,0.0,2.0,100.0,98.7,99.35,100.000000
...,...,...,...,...,...,...,...,...,...,...
8444,"[US, US, US, US, US, US]",1.0,0.000000,0.0,0.0,6.0,75.2,20.0,56.60,61.866667
8445,"[CL, CL]",0.0,0.000000,0.0,0.0,2.0,20.0,20.0,20.00,2.800000
8446,[US],1.0,0.000000,0.0,0.0,1.0,48.6,48.6,48.60,17.300000
8447,"[US, US]",1.0,0.000000,0.0,0.0,2.0,24.1,24.1,24.10,21.600000


In [15]:
df_all_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8449 entries, 0 to 8448
Data columns (total 32 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   org_name                                    8449 non-null   object        
 1   domain                                      8449 non-null   object        
 2   founded_on                                  8449 non-null   datetime64[ns]
 3   city                                        8449 non-null   object        
 4   category_list                               8383 non-null   object        
 5   category_groups_list                        8383 non-null   object        
 6   short_description                           8449 non-null   object        
 7   universities_of_founders                    8449 non-null   object        
 8   degrees_of_founders                         8256 non-null   object        
 9   subject_

In [16]:
# save to 'all_data.pkl'

save_datafram_flag  = True
if save_datafram_flag: 
    df_all_merged.to_pickle(r'Data/all_data.pkl')
    df_all_merged.to_excel(r'Data/all_data.xlsx', index = False)